In [0]:
%run /Workspace/Users/loganathansundaramsa@gmail.com/dataengg_inc_learning/databricks_learning/Brazilian_Ecommerce/gendral_config/com_utl_fun

In [0]:
import json

gendral_config = dbutils.notebook.run("/Workspace/Users/loganathansundaramsa@gmail.com/dataengg_inc_learning/databricks_learning/Brazilian_Ecommerce/gendral_config/common_notebook", 120)


config_dict = json.loads(gendral_config)

src = config_dict['src']
bronze = config_dict['bronze']
silver = config_dict['silver']
gold = config_dict['gold']
catalog = config_dict['CATALOG']
schema = config_dict['SCHEMA']

In [0]:
df_oitem = read_file(spark,"parquet",f"{bronze}/order_items")

In [0]:
schema_map = {
  "price": "decimal(10,2)",
  "shipping_limit_date": "timestamp",
  "freight_value": "decimal(10,2)"}

df_oitem = f_cast_col(df_oitem,schema_map)



In [0]:
df_oitem = deduplicate(df_oitem,["order_id","product_id","seller_id"])
df_oitem = drop_required_nulls(df_oitem,["order_id","product_id"])
df_oitem_filter = df_oitem.filter("try_cast(price as float) > 0 and try_cast(freight_value as float) > 0")
df_ord_itm = sum_two_columns(df_oitem_filter,"price","freight_value","item_total_value")
# df_sum.display()

In [0]:
df_sproducts = spark.read.table(f"{catalog}.{schema}.silver_products")
df_ssilvers  = spark.read.table(f"{catalog}.{schema}.silver_sellers")

df_check_prod = df_ord_itm.join(df_sproducts,df_ord_itm["product_id"] == df_sproducts["product_id"], how="semi")
df_check_silv = df_check_prod.join(df_ssilvers,df_check_prod["seller_id"] == df_ssilvers["seller_id"], how="semi")

# print(df_ord_itm.count(),df_check_prod.count(),df_check_silv.count())
# display(df_check_prod)

In [0]:
# Write data delta file /delta table in silver location

write_file(df_check_silv,f"{silver}/silver_order_items")

write_delta_liquid(df_check_silv, f"{catalog}.{schema}.silver_order_items",["seller_id"])